In [ ]:
!gdown --fuzzy https://drive.google.com/file/d/10UJTh0YUpVk75H2KMIiZ61sDsC-woWl0/view?usp=sharing


Downloading...
From (original): https://drive.google.com/uc?id=10UJTh0YUpVk75H2KMIiZ61sDsC-woWl0
From (redirected): https://drive.google.com/uc?id=10UJTh0YUpVk75H2KMIiZ61sDsC-woWl0&confirm=t&uuid=dec81147-9b6d-40eb-b38d-58dba8628e11
To: /content/temporal_ds.zip
100% 1.69G/1.69G [00:49<00:00, 34.0MB/s]


In [ ]:
%%capture
!unzip temporal_ds.zip

In [ ]:
%%capture
!pip install --upgrade torch torchvision torchaudio
!pip install pytorch_lightning
!pip install wandb


In [ ]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torchvision import transforms
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from torchmetrics.functional import accuracy
import glob
import random
import numpy as np
from torchmetrics import Accuracy, Precision, Recall
import wandb
from pytorch_lightning.loggers import WandbLogger

In [ ]:

class FireSeriesDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.transform = transform
        self.sets = glob.glob(f"{root_dir}/**/*")
        random.shuffle(self.sets)

    def __len__(self):
        return len(self.sets)

    def __getitem__(self, idx):
        img_folder = self.sets[idx]
        img_list = glob.glob(f"{img_folder}/*.jpg")

        labels = []
        for file in img_list:
            label_file = file.replace("images", "labels").replace(".jpg", ".txt")
            with open(label_file, "r") as f:
                lines = f.readlines()

            labels.append(np.array(lines[0].split(" ")[1:5]).astype("float"))

        labels = np.array(labels)
        xc = np.median(labels[:, 0]) * (0.9 + random.random()/5)
        yc = np.median(labels[:, 1]) * (0.9 + random.random()/5)
        wb = np.max(labels[:, 2]) * (0.9 + random.random()/5)
        hb = np.max(labels[:, 3]) * (0.9 + random.random()/5)

        x0 = xc - wb / 2
        y0 = yc - hb / 2
        x1 = xc + wb / 2
        y1 = yc + hb / 2

        tensor_list = []
        for file in img_list:
            im = Image.open(file)
            w, h = im.size
            cropped_image = im.crop(
                (int(x0 * w), int(y0 * h), int(x1 * w), int(y1 * h))
            )
            tensor_list.append(self.transform(cropped_image))

        return torch.cat(tensor_list, dim=0), int(img_folder.split("/")[-2])


In [ ]:

class FireDataModule(pl.LightningDataModule):
    def __init__(self, data_dir, batch_size=32, img_size=112):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.img_size = img_size

        self.transform = transforms.Compose([
          transforms.Resize((112, 112)),  # Resize to desired input size
          # transforms.RandomHorizontalFlip(p=0.3),  # Randomly flip horizontally with 30% probability
          # transforms.RandomRotation(degrees=5),  # Randomly rotate by up to 5 degrees
          # transforms.ColorJitter(brightness=0.05, contrast=0.05, saturation=0.05, hue=0.05),  # Slightly adjust color
          transforms.ToTensor(),  # Convert PIL image to tensor
          transforms.Normalize((0.5,), (0.5,)),
      ])

    def setup(self, stage=None):
        self.train_dataset = FireSeriesDataset(
            os.path.join(self.data_dir, "train"), transform=self.transform
        )
        self.val_dataset = FireSeriesDataset(
            os.path.join(self.data_dir, "val"), transform=self.transform
        )

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size)


In [ ]:

class FireClassifier(pl.LightningModule):
    def __init__(self, learning_rate=1e-4):
        super(FireClassifier, self).__init__()
        self.save_hyperparameters()

        # Using ResNet18 as the backbone
        self.model = models.resnet34(pretrained=True)

        # Modify the first convolutional layer to accept 12 channels instead of 3
        self.model.conv1 = nn.Conv2d(
            12, 64, kernel_size=7, stride=2, padding=3, bias=False
        )

        self.dropout = nn.Dropout(0.5)

        # Get the number of features for the last fully connected layer
        num_features = self.model.fc.in_features

        # Replace the last fully connected layer with a new one for binary classification
        self.model.fc = nn.Linear(num_features, 1)

        # Initialize the accuracy metric
        self.train_accuracy = Accuracy(task="binary")
        self.val_accuracy = Accuracy(task="binary")
        self.train_precision = Precision(task="binary")
        self.val_precision = Precision(task="binary")
        self.train_recall = Recall(task="binary")
        self.val_recall = Recall(task="binary")

    def forward(self, x):
        x = self.model(x)
        x = self.dropout(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x).squeeze()
        loss = F.binary_cross_entropy_with_logits(y_hat, y.float())
        acc = self.train_accuracy(torch.sigmoid(y_hat), y.int())
        precision = self.train_precision(torch.sigmoid(y_hat), y.int())
        recall = self.train_recall(torch.sigmoid(y_hat), y.int())
        self.log("train_loss", loss)
        self.log("train_acc", acc)
        self.log("train_precision", precision)
        self.log("train_recall", recall)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x).squeeze()
        loss = F.binary_cross_entropy_with_logits(y_hat, y.float())
        acc = self.val_accuracy(torch.sigmoid(y_hat), y.int())
        precision = self.val_precision(torch.sigmoid(y_hat), y.int())
        recall = self.val_recall(torch.sigmoid(y_hat), y.int())
        self.log("val_loss", loss)
        self.log("val_acc", acc)
        self.log("val_precision", precision)
        self.log("val_recall", recall)
        return loss

    def configure_optimizers(self):
      optimizer = torch.optim.Adam(self.parameters(), lr=self.hparams.learning_rate, weight_decay=1e-4)  # Added weight_decay
      scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.2, patience=5, min_lr=1e-6)
      return {
          'optimizer': optimizer,
          'lr_scheduler': {
              'scheduler': scheduler,
              'monitor': 'val_loss'
          }
      }



In [ ]:

# Initialize the DataModule
data_dir = "temporal_ds/images"
data_module = FireDataModule(data_dir)

# Initialize the model
model = FireClassifier()

# Define callbacks
checkpoint_callback = ModelCheckpoint(monitor="val_acc", mode="max", save_top_k=1)

# Initialize WandbLogger
wandb_logger = WandbLogger(project='fire_detection_project')

# Initialize the Trainer
trainer = pl.Trainer(
    max_epochs=20,
    callbacks=[checkpoint_callback],
    logger=wandb_logger
)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 148MB/s]
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.uti

In [ ]:
# Train the model
trainer.fit(model, data_module)
wandb.finish()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name            | Type            | Params | Mode 
------------------------------------------------------------
0 | model           | ResNet          | 21.3 M | train
1 | dropout         | Dropout         | 0      | train
2 | train_accuracy  | BinaryAccuracy  | 0      | train
3 | val_accuracy    | BinaryAccuracy  | 0      | train
4 | train_precision | BinaryPrecision | 0      | train
5 | val_precision   | BinaryPrecision | 0      | train
6 | train_recall    | BinaryRecall    | 0      | train
7 | val_recall      | BinaryRecall    | 0      | train
------------------------------------------------------------
21.3 M    Trainable params
0         Non-trainable params
21.3 M    Total params
85.254    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [ ]:
wandb.finish()